In [ ]:
import pickle 
import numpy as np
import pandas as pd
import ipyvuetify as v

import warnings
warnings.filterwarnings("ignore")
from thermo import Joback

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

In [ ]:
def nbp_calc(smiles, family):
    with open('hasher.pkl', 'rb') as file:
        fh = pickle.load(file=file)
    
    with open('params_Final.pkl', 'rb') as file:
        min_, max_ = pickle.load(file)

    model = tf.keras.models.load_model('PINN_Final.h5')

    smiles_code = smiles
    try:
        J = Joback(smiles_code).counts
    except:
        return -1000

    fga = np.zeros(40)
    for key in J:
        fga[key-1] = J[key]

    family_val = fh.transform(np.array([family])).toarray()

    X = np.array(family_val.tolist()[0] + fga.tolist())
    x = (X-min_[:-1])/(max_[:-1] - min_[:-1]) 
    
    yp = model(x.to_numpy()[None, :])[0]
    Yp = yp*(max_[-1] - min_[-1]) + min_[-1]
    
    return Yp[0].numpy()


In [ ]:
df_smiles = pd.read_csv('smf.csv')
df_smiles.drop(columns='Unnamed: 0', inplace=True)
df_family = df_smiles.drop_duplicates(subset='Family')[['Family']]

In [ ]:
def guess_family(item, *args):
    if item.v_model is None:
        val = ['']
    elif type(item.v_model) == list:
        val = item.v_model[0]
    else:
        val = item.v_model

    if val in df_smiles['smiles'].values:
        family_field.v_model = df_smiles.loc[df_smiles['smiles']==val]['Family'].to_numpy().tolist()
    
    update()
    
def update(*args):
    if ( (len(smiles_field.v_model) >= 1) + (len(family_field.v_model) >=1) ) == 2:
        predict_btn.disabled = False
    else:
        predict_btn.disabled = True
        
def predict(item, *args):
    
    predict_btn.loading = True
    
    user_smiles = smiles_field.v_model
    user_family = family_field.v_model[0]
    
    if type(user_smiles) == list:
        user_smiles = user_smiles[0]
        
    nbp = nbp_calc(smiles=user_smiles, family=user_family)
    
    if nbp == -1000:
        results.children = ["SMILES Parsing Error! Please enter a valid SMILES value."]
    else:
        results.children = [f'Normal Boiling Point = {nbp-273.15 :.2f} degC']
        
    predict_btn.loading = False

smiles_field = v.Combobox(label='Smiles', items=df_smiles[['smiles']].to_numpy().tolist(), v_model=[], class_='mx-4', dark=True, hide_details=True, hide_no_data=True, counter_value=5)
family_field = v.Autocomplete(label='Family', items=df_family[['Family']].to_numpy().tolist(), v_model=[], class_='mx-4', dark=True, hide_no_data=True)
predict_btn = v.Btn(children=['Predict'], color='orange lighten-1', dark=True, disabled=True)
results = v.Text(children=[''], class_='align-right mt-2')

smiles_field.on_event('change', guess_family)
family_field.on_event('change', update)
predict_btn.on_event('click', predict)

selection_row = v.Row(children=[smiles_field, family_field])
action_row = v.Row(children=[predict_btn, results], class_='mx-1 pb-2 justify-space-between')

title = v.CardTitle(children=['Thermo-PINN'])
app_card = v.Card(children=[title, selection_row, action_row], class_='pa-4 ma-4', dark=True, style_='max-width:600px')

app = v.App(children=[app_card])
app